In [1]:
import os
import cv2
from tqdm import tqdm

def convert_wider_to_yolo(wider_annotation_path, images_root_dir, output_labels_dir):
    """
    Versi Robust: Mencari baris berakhiran .jpg agar tidak crash saat ada baris kosong/error.
    """
    if not os.path.exists(output_labels_dir):
        os.makedirs(output_labels_dir)

    print(f"📂 Membaca file anotasi: {wider_annotation_path}")
    
    with open(wider_annotation_path, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()] # Hapus baris kosong

    i = 0
    pbar = tqdm(total=len(lines))
    
    while i < len(lines):
        line = lines[i]
        
        # --- LOGIKA JANGKAR (ANCHOR) ---
        # Cari baris yang berakhiran .jpg atau .png. Ini pasti Header File.
        if not (line.endswith('.jpg') or line.endswith('.png')):
            # Jika bukan file gambar, skip baris ini (mungkin sampah/koordinat sisa)
            i += 1
            pbar.update(1)
            continue
            
        # Jika sampai sini, berarti 'line' adalah nama file
        file_path = line
        i += 1 # Pindah ke baris berikutnya (seharusnya jumlah wajah)
        
        if i >= len(lines): break

        # Coba baca jumlah wajah
        try:
            num_faces = int(lines[i])
            i += 1 # Pindah ke baris koordinat pertama
        except ValueError:
            # Jika baris setelah nama file BUKAN angka, berarti file korup. Skip.
            print(f"⚠️ Warning: Format salah setelah file {file_path}")
            continue

        # Update progress bar manual biar pas
        pbar.update(2) 

        # Cek Validitas Gambar
        img_full_path = os.path.join(images_root_dir, file_path)
        
        # Kita perlu baca dimensi gambar untuk normalisasi YOLO
        img = None
        if os.path.exists(img_full_path):
            img = cv2.imread(img_full_path)
        
        # Jika gambar tidak ada/rusak, kita tetap harus SKIP baris koordinat di txt
        # agar loop berikutnya tidak error membaca koordinat sebagai nama file.
        if img is None:
            # print(f"⚠️ Skip (Gambar Missing): {file_path}")
            i += num_faces
            pbar.update(num_faces)
            continue

        img_h, img_w = img.shape[:2]
        yolo_lines = []
        
        # --- BACA KOORDINAT WAJAH ---
        for _ in range(num_faces):
            if i >= len(lines): break
            
            coord_line = lines[i].split()
            
            # Pastikan baris ini punya cukup angka (WIDER format min 4 angka: x1, y1, w, h)
            if len(coord_line) >= 4:
                try:
                    x1 = float(coord_line[0])
                    y1 = float(coord_line[1])
                    w = float(coord_line[2])
                    h = float(coord_line[3])
                    
                    # Validasi w/h tidak 0
                    if w > 0 and h > 0:
                        # Rumus Konversi YOLO (Normalized)
                        cx = (x1 + w / 2) / img_w
                        cy = (y1 + h / 2) / img_h
                        nw = w / img_w
                        nh = h / img_h
                        
                        # Clipping agar tidak lebih dari 1.0 (kadang label widerface keluar gambar)
                        cx = max(0, min(1, cx))
                        cy = max(0, min(1, cy))
                        nw = max(0, min(1, nw))
                        nh = max(0, min(1, nh))

                        yolo_lines.append(f"0 {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")
                except ValueError:
                    pass # Skip baris koordinat error

            i += 1
        
        pbar.update(num_faces)
        
        # --- SIMPAN LABEL ---
        if yolo_lines:
            category_folder = os.path.dirname(file_path)
            output_subdir = os.path.join(output_labels_dir, category_folder)
            
            if not os.path.exists(output_subdir):
                os.makedirs(output_subdir)
                
            filename_txt = os.path.basename(file_path).replace('.jpg', '.txt').replace('.png', '.txt')
            save_path = os.path.join(output_subdir, filename_txt)
            
            with open(save_path, 'w') as f_out:
                f_out.write('\n'.join(yolo_lines))

    print("\n✅ Konversi Selesai! Struktur file .txt aman.")

# --- KONFIGURASI JALUR FOLDER (SESUAIKAN DENGAN STRUKTURMU) ---
# Gunakan Absolute Path agar aman dari error "Double Path"
WIDER_TXT = os.path.join(r"D:\1. Kuliah\Semester 7\test_code\YOLO26\datasets\face_split\wider_face_val_bbx_gt.txt")
IMAGES_DIR = os.path.join(r"D:\1. Kuliah\Semester 7\test_code\YOLO26\datasets\val\images")
LABELS_OUTPUT = os.path.join(r"D:\1. Kuliah\Semester 7\test_code\YOLO26\datasets\val\labels")

if __name__ == "__main__":
    # Cek dulu sebelum jalan, biar gak error kaget lagi
    if not os.path.exists(WIDER_TXT):
        print(f"❌ ERROR: File txt tidak ditemukan di:\n{WIDER_TXT}")
        print("👉 Coba cek lagi lokasinya pakai File Explorer.")
    elif not os.path.exists(IMAGES_DIR):
        print(f"❌ ERROR: Folder images tidak ditemukan di:\n{IMAGES_DIR}")
    else:
        print("✅ Jalur ditemukan! Memulai konversi...")
        convert_wider_to_yolo(WIDER_TXT, IMAGES_DIR, LABELS_OUTPUT)

✅ Jalur ditemukan! Memulai konversi...
📂 Membaca file anotasi: D:\1. Kuliah\Semester 7\test_code\YOLO26\datasets\face_split\wider_face_val_bbx_gt.txt


100%|██████████| 46160/46160 [01:35<00:00, 484.26it/s] 


✅ Konversi Selesai! Struktur file .txt aman.
